In [5]:
import tweepy


# Put your unique token from twitter
token = "AAAAAAAAAAAAAAAAAAAAACRHhAEAAAAABCEus62dFo1plpi5zCURvv2hKow%3DR0CR1hFimLkwaFYm4Y8CdPSDQyv1lwtvZlLl9fnSF5mcl5pMWh"

client = tweepy.Client(bearer_token=token)


# User Id from twitter profile
userid = "1339518164410003457"
# This is a get for the tweets from userid
tweets = client.get_users_tweets(id=userid)
maxResult = 10

paginator = tweepy.Paginator(client.get_users_tweets, id=userid, max_results=maxResult)

#flatten is needed to loop trough the paginator
for tweet in paginator.flatten(limit=maxResult):

    print(tweet)

RT @djeppink: Deze week mijn interview in ⁦@ewmagazinenl⁩. De Tweede Kamer: beter minder boosheid theater, en meer realistische oplossingen…
Lees hier het hele interview met @DJEppink: 
https://t.co/t6B3WFcgkZ (2/2)
"Op televisie lijkt het of iedereen hier de hele tijd boos is. Dat heb ik niet. Ik ben niet boos en leg veel goede contacten. Ik drink koffie met de heer Wilders en de heer Kuzu, en iedereen die daartussen zit." 

Lang interview in @EWmagazinenl met @DJEppink. 
#Leestip (1/2) https://t.co/h2mKT4UKqh
RT @djeppink: In marge Oekraïne oorlog, laait ander conflict op. Azerbeidzjan lanceert aanval op Armenië. Is een kwalijke zaak. Azeri bewin…
RT @MPHoogeveen: De renteverhoging van de #ECB is too little, too late. Het gericht opkopen van staatsschulden gaat door. We zitten in een…
Dit kabinet wil de nieuwe #pensioenwet met spoed doordrukkken, terwijl er nog vele zorgen en vragen zijn. #JA21 vindt dat het beste #pensioenstelsel ter wereld alleen mag worden herzien, als het alterna

In [ ]:
import tweepy

from deep_translator import (GoogleTranslator,
                             MicrosoftTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                             PapagoTranslator,
                             DeepL,
                             QCRI,
                             single_detection,
                             batch_detection)


# Put your unique token from twitter
token = "%"

client = tweepy.Client(bearer_token=token)


# User Id from twitter profile
userid = "1339518164410003457"
# This is a get for the tweets from userid
tweets = client.get_users_tweets(id=userid)
maxResult = 10

paginator = tweepy.Paginator(client.get_users_tweets, id=userid, max_results=maxResult)

#flatten is needed to loop trough the paginator
for tweet in paginator.flatten(limit=maxResult):

    print(tweet)
